---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def formatName(x):
    if (x['RawData'].find('[edit]') > 0):
        x['State'] = x['RawData'].split('[')[0].strip()
        x['RegionName'] = np.NaN
    else:
        x['State'] = np.NaN
        x['RegionName'] = x['RawData'].split(' (')[0].strip()
    return x

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    
    df = pd.read_csv('university_towns.txt', names=['RawData', 'State', 'RegionName'])
    df.apply(formatName, axis=1)
    df['State'] = df['State'].fillna(method='ffill')
    df = df.dropna().reset_index()
    return df[['State','RegionName']]

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', 0, skiprows=219, index_col=None, parse_cols=[4,6], names=[['Quarter','GDP']])
    df['prev GDP'] = df['GDP'].shift(1)
    df['next GDP'] = df['GDP'].shift(-1)
    df['diff1'] = df['GDP'] - df['prev GDP']
    df['diff2'] = df['next GDP'] - df['GDP']
    df['start recession'] = (df['diff1'] < 0) & (df['diff2'] < 0)
    rec_start = df[df['start recession']].iloc[0]['Quarter']
    return rec_start

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', 0, skiprows=219, index_col=None, parse_cols=[4,6], names=[['Quarter','GDP']])
    df['prev GDP'] = df['GDP'].shift(1)
    df['prev2 GDP'] = df['GDP'].shift(2)
    df = df[df['Quarter'] >= get_recession_start()]
    df['diff1'] = df['GDP'] - df['prev GDP']
    df['diff2'] = df['prev GDP'] - df['prev2 GDP']
    df['end recession'] = (df['diff1'] > 0) & (df['diff2'] > 0)
    rec_end = df[df['end recession']].iloc[0]['Quarter']
    return rec_end

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', 0, skiprows=219, index_col=None, parse_cols=[4,6], names=[['Quarter','GDP']])
    df = df[df['Quarter'] >= get_recession_start()]
    df = df[df['Quarter'] <= get_recession_end()]
    bottom = df[df['GDP'] == np.min(df['GDP'])].iloc[0]['Quarter']
    return bottom

get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df['State'] = df.apply(lambda x: states[x['State']], axis=1)
    df = df.set_index(['State',"RegionName"])
    quarters = []
    
    for year in range(2000,2017):
        #q1
        quarter = str(year)+'q1'
        rows = [str(year)+'-01',str(year)+'-02',str(year)+'-03']
        df[quarter] = df.apply(lambda x: np.mean(x[rows]), axis=1)
        quarters.append(quarter)
        #q2
        quarter = str(year)+'q2'
        rows = [str(year)+'-04',str(year)+'-05',str(year)+'-06']
        df[quarter] = df.apply(lambda x: np.mean(x[rows]), axis=1)
        quarters.append(quarter)
        #q3
        quarter = str(year)+'q3'
        rows = [str(year)+'-07',str(year)+'-08',str(year)+'-09']
        df[quarter] = df.apply(lambda x: np.mean(x[rows]), axis=1)
        quarters.append(quarter)
        #q4
        quarter = str(year)+'q4'
        if (year != 2016):
            rows = [str(year)+'-10',str(year)+'-11',str(year)+'-12']
            df[quarter] = df.apply(lambda x: np.mean(x[rows]), axis=1)
            quarters.append(quarter)
    
    return df[quarters]

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # get quarter_before_recession
    gdp = pd.read_excel('gdplev.xls', 0, skiprows=219, index_col=None, parse_cols=[4,6], names=[['Quarter','GDP']])
    gdp['next GDP'] = gdp['GDP'].shift(-1)
    gdp['next2 GDP'] = gdp['GDP'].shift(-2)
    gdp['diff1'] = gdp['next GDP'] - gdp['GDP']
    gdp['diff2'] = gdp['next2 GDP'] - gdp['next GDP']
    gdp['quarter_before_recession'] = (gdp['diff1'] < 0) & (gdp['diff2'] < 0)
    quarter_before_recession = gdp[gdp['quarter_before_recession']].iloc[0]['Quarter']
    
    rec_bottom = get_recession_bottom()
    
    housingData = convert_housing_data_to_quarters()
    uTown = get_list_of_university_towns()
    uTown['isUTown'] = "Yes"
    allData = pd.DataFrame(columns=[quarter_before_recession, rec_bottom])
    allData[quarter_before_recession] = housingData[quarter_before_recession]
    allData[rec_bottom] = housingData[rec_bottom]
    allData['ratio'] = allData[quarter_before_recession].div(allData[rec_bottom])

    allData = pd.merge(uTown, allData, how='right', left_on=["State","RegionName"], right_index=True)
    allData['isUTown'] = allData['isUTown'].fillna('No')
    allData = allData.dropna()

    uTownData = allData[allData['isUTown'] == 'Yes']
    nonUTownData = allData[allData['isUTown'] == 'No']
    
    tTestResult = ttest_ind(uTownData['ratio'], nonUTownData['ratio'])
    pValue = tTestResult.pvalue
    different = (pValue < 0.01)
    uTownMean = np.mean(uTownData['ratio'])
    nonUTownMean = np.mean(nonUTownData['ratio'])
    if(uTownMean < nonUTownMean):
        better = "university town"
    else:
        better = "non-university town"
    return (different, pValue, better)
